# Loading word2vec data and preprocessing


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')


import pandas as pd
from keras.layers import  Flatten, Dense, Dropout


In [ ]:
train=pd.read_csv("/content/drive/My Drive/NNFL Fall'19 Project/data_7000_new.csv",header=None)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
import cv2

import pandas as pd

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
import os
images={}
i=0
# print(train[0][100])
# a=[]
not_aval=[]
for file in train[0]:
  a=cv2.imread(os.path.join("/content/drive/My Drive/NNFL Fall'19 Project/data_7000/", file))
  print(i)
  if a is not None:
    images[i] = cv2.resize(a,(240,240))
    # print(a)
    i+=1
  else:
    print(file,i)
    i+=1
    not_aval.append(i-1)

  

In [ ]:
print(train[0][not_aval])
trainFinal=train.drop(not_aval,axis=0)

In [ ]:
trainFinal.shape

In [ ]:
x_text=trainFinal[3]
x_text.shape


In [ ]:
y3=trainFinal[6]
y4=trainFinal[7]
y5=trainFinal[8]


In [ ]:

a3={}
a4={}
a5={}

for i in y3:
  if i in a3.keys():
    a3[i]+=1
  else:
    a3[i]=1
for i in y4:
  if i in a4.keys():
    a4[i]+=1
  else:
    a4[i]=1
for i in y5:
  if i in a5.keys():
    a5[i]+=1
  else:
    a5[i]=1


In [ ]:
y4f= y4.replace({"motivational":+1, "not_motivational":0,'negative':None,'neutral':None,'positive':None,'very_negative':None,'very_positive':None})
a4f={}
for i in y4f:
  if i in a4f.keys():
    a4f[i]+=1
  else:

    a4f[i]=1

In [ ]:
x_text=x_text.astype(str)


In [ ]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
word_index=[]
num_words = 12880
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ')
tokenizer.fit_on_texts(x_text.values)
X = tokenizer.texts_to_sequences(x_text.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 50
X = pad_sequences(X, maxlen=max_length_of_text)

print(word_index)


In [ ]:
import numpy as np
embeddings_index = {}
f = open( 'glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
moti_Text=X[y4f.notnull()]
moti_images=pd.Series(images)[y4f.notnull()]
y4f=y4f[y4f.notnull()]


# Creating 3 diffrent models

# Motivational or Not

In [ ]:
import keras
from keras.layers import BatchNormalization as BatchNorm



import keras
from keras.layers import Convolution1D as Conv1D
from  keras.layers import MaxPooling1D as MaxPooling1D
embed_dim = 200 #Change to observe effects
lstm_out = 512 #Change to observe effects
batch_size = 32


input1 = keras.layers.Input(shape=(240,240,3,),name="image_input")
input2 = keras.layers.Input(shape=(50,),name="text_input")

iml=Conv2D(64, (3, 3), activation='relu',name="Conv2D_1")(input1)
iml=MaxPool2D(pool_size=(2, 2),name='MaxPool2D_1')(iml)
iml=Conv2D(64, (3, 3), activation='relu',name="Conv2d_2")(iml)
iml=MaxPool2D(pool_size=(2, 2),name="MaxPool2D_2")(iml)

iml=Conv2D(128, (3,3), activation='relu',name="Conv2D_3")(iml)
iml=MaxPool2D(pool_size=(2, 2),name="MaxPool2D_3")(iml)

iml = Flatten(name="Flatten_images")(iml)

x = Embedding(12881, embed_dim,weights=[embedding_matrix],name="Embedding_layer")(input2)  
x = Conv1D(512, 5, activation='relu', kernel_initializer='normal',name="Conv1D_1")(x)
x = Conv1D(256, 5, activation='relu', kernel_initializer='normal',name="Conv1D_2")(x)
x = MaxPooling1D(5,name="MaxPooling1D_text")(x)
x = Flatten(name="Flatten_text")(x)
added=keras.layers.Concatenate(axis=-1)([x,iml])
x= Dense(128,activation='relu', kernel_initializer='normal',name="Dense_1")(added)
preds = Dense(1, activation='sigmoid', kernel_initializer='normal',name="Dense_2")(x)

model = Model(inputs=[input1, input2], outputs=preds)



In [ ]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
y4final=y4f
moti_train_text,moti_test_text,moti_train_image,moti_test_image, y_moti_train, y_moti_test = train_test_split(moti_Text,moti_images,y4final,test_size = 0.1)


In [ ]:
model.summary()

In [ ]:
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras import optimizers
model.compile(loss = 'binary_crossentropy', optimizer='sgd', metrics = ['mae','accuracy',f1_m])

In [ ]:

moti_train_image=np.array(moti_train_image.tolist())
moti_test_image=np.array(moti_test_image.tolist())
moti_train_image=np.true_divide(moti_train_image,255)
moti_test_image=np.true_divide(moti_test_image,255)



In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow( np.multiply(moti_train_image[0],255));


In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement.hdf5"


checkpoint = ModelCheckpoint(filepath, monitor='val_f1_m', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(x=[moti_train_image,moti_train_text], y=y_moti_train, batch_size=32, epochs=50,callbacks=callbacks_list, verbose=1,validation_data=([moti_test_image,moti_test_text],y_moti_test))

In [ ]:
model.load_weights("weights-improvement.hdf5")

In [ ]:
y_predicted=model.predict([moti_test_image,moti_test_text])
y_predicted=y_predicted.reshape(-1)

In [ ]:
ans=[]
for i in y_predicted:
  if i>=.5:
    ans.append(1)
  else:
    ans.append(0)
print(ans)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
y5f= y5.replace({'negative': 1,'neutral': 2,'positive': 3,'positivechandler_Friday-Mood-AF.-meme-Friends-ChandlerBing.jpg':3,'very_negative': 0,'very_positive': 4})
a5f={}
for i in y5f:
  if i in a5f.keys():
    a5f[i]+=1
  else:
    a5f[i]=1
print(a5f)

y3f= y3.replace({'hateful_offensive': 3,'very_offensive': 2,'slight': 1,'not_offensive': 0,'very_positive':None,'motivational': None,'not_motivational':None,'positive':None})
y3fmean=y3f.mean()
# y3f=y3f.fillna(y3fmean)
a3f={}
for i in y3f:
  if i in a3f.keys():
    a3f[i]+=1
  else:
    a3f[i]=1
print(a3f)

# offensive or not

In [ ]:
pos_text=X[y5f.notnull()]
pos_images=pd.Series(images)[y5f.notnull()]
y5f=y5f[y5f.notnull()]

offensive_Text=X[y3f.notnull()]
offensive_images=pd.Series(images)[y3f.notnull()]
y3f=y3f[y3f.notnull()]

off_train_text,off_test_text,off_train_image,off_test_image, y3_off_train, y3_off_test = train_test_split(offensive_Text,offensive_images,y3f,test_size = 0.1)
pos_train_text,pos_test_text,pos_train_image,pos_test_image, y5_pos_train, y5_pos_test = train_test_split(pos_text,pos_images,y5f,test_size = 0.1)

In [ ]:
off_train_image=np.array(off_train_image.tolist())
off_test_image=np.array(off_test_image.tolist())
off_train_image=np.true_divide(off_train_image,255)
off_test_image=np.true_divide(off_test_image,255)

In [ ]:

import keras
from keras.layers import BatchNormalization as BatchNorm
embed_dim = 200 #Change to observe effects
lstm_out = 512 #Change to observe effects
batch_size = 32


input1 = keras.layers.Input(shape=(240,240,3,))
input2 = keras.layers.Input(shape=(50,))

iml=Conv2D(64, (3, 3), activation='relu')(input1)
iml=MaxPool2D(pool_size=(2, 2))(iml)

iml=Conv2D(64, (3, 3), activation='relu')(iml)
iml=MaxPool2D(pool_size=(2, 2))(iml)

iml=Conv2D(128, (3,3), activation='relu')(iml)
iml=MaxPool2D(pool_size=(2, 2))(iml)


iml = Flatten()(iml)



import keras
from keras.layers import Convolution1D as Conv1D
from  keras.layers import MaxPooling1D as MaxPooling1D
x = Embedding(12881, embed_dim,weights=[embedding_matrix])(input2)  
x = Conv1D(512, 5, activation='relu', kernel_initializer='normal')(x)
x = Conv1D(256, 5, activation='relu', kernel_initializer='normal')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu', kernel_initializer='normal')(x)
x = Flatten()(x)

added=keras.layers.Concatenate(axis=-1)([x,iml])
x=Dropout(0.1)(x)
x= Dense(128,activation='relu', kernel_initializer='normal')(added)

preds = Dense(1, activation='linear', kernel_initializer='normal')(x)




model = Model(inputs=[input1, input2], outputs=preds)

import keras


In [ ]:

model.compile(loss = 'mean_squared_error', optimizer='sgd', metrics = ['mae','accuracy'])

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):

  def __init__(self, valid_data, *args, **kwargs):
    self.valid_data = valid_data

    super(Metrics, self).__init__(*args, **kwargs)

  def on_train_begin(self, logs={}):
    pass
  
  def on_epoch_end(self, epoch, logs={}):
    global temp
    val_predict =self.model.predict(self.valid_data[0])
    val_targ =self.valid_data[1]
    
    _val_f1 = f1_score(val_targ, np.round(val_predict), average = 'macro')
    print(" — val_macro_f1: {}".format(_val_f1))
    return

metric = Metrics(([off_test_image,off_test_text],y3_off_test))

In [ ]:

from keras.callbacks import ModelCheckpoint
filepath="weights-improvement"

checkpoint = ModelCheckpoint('weights.{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint,metric]

model.fit(x=[off_train_image,off_train_text], y=y3_off_train, batch_size=32, epochs=30,callbacks=callbacks_list, verbose=1,validation_data=([off_test_image,off_test_text],y3_off_test))

# positive or negative

In [ ]:
pos_train_image=np.array(pos_train_image.tolist())
pos_test_image=np.array(pos_test_image.tolist())
pos_train_image=np.true_divide(pos_train_image,255)
pos_test_image=np.true_divide(pos_test_image,255)






In [ ]:

import keras
from keras.layers import BatchNormalization as BatchNorm
embed_dim = 200 #Change to observe effects
lstm_out = 512 #Change to observe effects
batch_size = 32


input1 = keras.layers.Input(shape=(240,240,3,))
input2 = keras.layers.Input(shape=(50,))

iml=Conv2D(64, (3, 3), activation='relu')(input1)
iml=MaxPool2D(pool_size=(2, 2))(iml)

iml=Conv2D(64, (3, 3), activation='relu')(iml)
iml=MaxPool2D(pool_size=(2, 2))(iml)

iml=Conv2D(128, (3,3), activation='relu')(iml)
iml=MaxPool2D(pool_size=(2, 2))(iml)


iml = Flatten()(iml)



import keras
from keras.layers import Convolution1D as Conv1D
from  keras.layers import MaxPooling1D as MaxPooling1D
x = Embedding(12881, embed_dim,weights=[embedding_matrix])(input2)  
x = Conv1D(512, 5, activation='relu', kernel_initializer='normal')(x)
x = Conv1D(256, 5, activation='relu', kernel_initializer='normal')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu', kernel_initializer='normal')(x)
x = Flatten()(x)

added=keras.layers.Concatenate(axis=-1)([x,iml])
x= Dense(128,activation='relu', kernel_initializer='normal')(added)
preds = Dense(1, activation='linear', kernel_initializer='normal')(x)




model = Model(inputs=[input1, input2], outputs=preds)

import keras


In [ ]:
model.compile(loss = 'mae', optimizer="sgd", metrics = ['mae','accuracy'])

In [ ]:



from keras.callbacks import ModelCheckpoint
filepath="weights-improvement.hdf5"


checkpoint = ModelCheckpoint(filepath, monitor='val_mean_absolute_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(x=[pos_train_image,pos_train_text], y=y5_pos_train, batch_size=32, epochs=120,callbacks=callbacks_list, verbose=1,validation_data=([pos_test_image,pos_test_text],y5_pos_test))

In [ ]:
 min(model.predict([pos_train_image,pos_train_text]))

In [ ]:
np.max(y5_pos_train)